In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import os
import json
import torch
import yaml
import numpy as np
import cv2
import copy
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from nerfstudio.utils.eval_utils import eval_setup
from inerf.inerf_utils import eval_image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
os.chdir('/workspace')
MODEL_PATH = "/workspace/outputs/jackal_training_data_5/plane-nerf/2024-01-14_133459"
EVAL_PATH = "/stored_data/jackal_evaluation_data"

TRANSFORM_PATH = os.path.join(EVAL_PATH, "transforms.json")
with open(TRANSFORM_PATH) as f:
    transform = json.load(f)


In [4]:
config_path = os.path.join(MODEL_PATH, "config.yml")
config, pipeline, checkpoint_path, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )

WARNING: Using slower TCNN CutlassMLP instead of TCNN FullyFusedMLP

Use layer width of 16, 32, 64, or 128 to use the faster TCNN FullyFusedMLP.

WARNING: Using slower TCNN CutlassMLP instead of TCNN FullyFusedMLP

Use layer width of 16, 32, 64, or 128 to use the faster TCNN FullyFusedMLP.

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/jackal_training_data_5/plane-nerf/2024-01-14_133459/nerfstudio_models/step-000009999.ckpt

In [6]:
eval_image(pipeline,transform)

Setting up training dataset...

Caching all 500 images.

images/111.png

Output()

images/356.png

...

images/302.png

images/166.png

images/237.png

images/169.png

FileNotFoundError: [Errno 2] No such file or directory: 'images/452.png'